In [7]:
import numpy as np
from numpy import array

from colorama import Fore, Style, Back


# If a file is defined, we try to load it, redefining boardsize and squaresize


# If no file, we use a predefined embed game

# Define the Basic square size. Doing so, it defines easily the BOARDSize (squaresize **2)
#Board = array([
#        [3,0,4,0],
#        [0,1,0,0],
#        [0,0,2,0],
#        [0,3,0,4]])

# 4x4 Normal one
#Board = array([
#        [8,0,0,5,6,0,0,7,2],
#        [7,2,0,1,0,0,9,0,0],
#        [0,1,6,0,7,0,0,0,0],
#        [0,0,0,2,0,0,3,0,0],
#        [0,8,9,7,3,4,5,6,0],
#        [0,0,4,0,0,1,0,0,0],
#        [0,0,0,0,9,0,6,1,0],
#        [0,0,1,0,0,3,0,4,7],
#        [3,6,0,0,1,7,0,0,9]
#        
#    ])

# 4x4 Evil One
Board = array([
        [0,5,0,0,0,0,6,2,0],
        [0,0,0,2,0,0,7,0,0],
        [1,0,0,4,0,0,8,9,0],
        [0,0,0,0,2,0,9,0,6],
        [0,8,0,3,4,6,0,5,0],
        [6,0,1,0,9,0,0,0,0],
        [0,4,6,0,0,3,0,0,9],
        [0,0,7,0,0,2,0,0,0],
        [0,9,8,0,0,0,0,7,0]        
    ])

# 16x16
#Board = array([
#        [ 8, 7, 0, 0,  0, 0, 0, 0,   0, 3, 0, 0,   13, 0, 4, 0],
#        [ 0, 5,14, 0,  0, 0, 3,10,  15, 9, 1, 0,    0, 6, 0, 0],
#        [16, 0, 0, 0,  5, 8, 7, 0,   0,14, 0, 0,    9, 0,11,12],
#        [ 0, 0, 4, 0,  0,14, 6,13,   0,11,10,12,    0, 7, 0, 3],
#        
#        [14, 0, 0, 8,  0, 0, 1, 0,   0, 0, 0, 3,    7, 4,12, 0],
#        [ 9, 0, 0, 0,  0, 6,15,12,   0, 0,13,14,    0, 3, 1, 0],
#        [11, 0,10, 3,  0, 0,13, 0,   0, 8, 0, 1,    0, 0, 6, 0],
#        [6,  0, 0, 1, 14, 0, 4, 0,   0, 5, 0, 9,   11, 0, 0,13],
#        
#        [ 0, 0, 0, 0, 15, 0, 0, 0,   0, 0, 9, 0,    5, 0, 2,10],
#        [10, 1, 0, 0,  6, 0, 5, 0,  13,15, 7,16,    0, 0, 0, 0],
#        [ 0, 0,16,11,  0, 4, 0, 8,   2, 0, 0, 0,    0,13, 0, 7],
#        [ 0, 9, 0, 7,  1, 3, 0, 2,   6, 0, 8,10,   16,15,14, 4],
#        
#        [ 7, 0,13, 0,  9,16, 0, 5,   0, 0,14, 4,    3, 8, 0, 2],
#        [ 0, 0, 3, 0, 10, 0, 0, 0,   0, 0, 0, 0,    0,16,15, 0],
#        [ 1, 0, 9, 0,  0, 0,14, 4,   0, 0, 0, 0,    0, 0, 7, 0],
#        [ 0, 6, 8, 0,  3, 0, 0, 0,  10, 7, 0, 0,    0, 0, 0, 0]
#        
#    ])

DEBUG = False
THINKING = False

def DisplayBoard(B, O = array([])):
    def printSeparatorLine(sep='-'):
        # print a line
        print('||', end='')
        if ( i % SquareSize == 0 ):
            sep = '='
        for z in range(B.shape[1]*4 -2 + SquareSize):
            print(sep, end='')
        print('||')        

    # No Original board ? Copy the other one
    if O.size == 0:
        O = B.copy()
        
    print('')
    for i in range(B.shape[0]):
        printSeparatorLine()
        for j in range(B.shape[1]):
            if ( j % SquareSize == 0 ):
                print('|', end='')
            if ( B[i,j] == 0 ):
                print('|',' ','', end='')
            else:
                if B[i,j] == O[i,j]:
                    print('|',B[i,j],'', end='')

                else:
                    print(Style.BRIGHT, end='')
                    print(Fore.CYAN, end='')
                    print('|',B[i,j],'', end='')
                    print(Fore.RESET, end='')
                    print(Style.RESET_ALL, end='')
                    
        print('|| \n', end='')
    printSeparatorLine('=')
    
def Solve(B):
    def debug(msg):
        if DEBUG:
            print(msg)
            
    def getSquareIndices(i,j):
        sqind = np.array([], int)
        sqind = np.append(sqind, (i // SquareSize)*SquareSize)
        sqind = np.append(sqind, (j // SquareSize)*SquareSize)
        debug('sqind : {}'.format(sqind))

        debug('working in square {} {}'.format(sqind[0], sqind[1]))
        return sqind
    
    def getPossibles(i,j):
        poss = np.array([], int);
        # search all values iteratively in the line
        for val in range(1,BoardLength+1):
            found = False;
            debug('Looking for {} FROM {} {}'.format(val, i, j))
            # Horizontal and Vertical checking
            for ind in range(BoardLength):
                if(B[i,ind] == val):
                    debug('found {} in a line at pos {} {}'.format(val, i, ind))
                    found = True
                if(B[ind,j] == val):
                    debug('found {} in a col at pos {} {}'.format(val, j, ind))
                    found = True
            
            # SquareChecking
            sqind = getSquareIndices(i,j)
            for ind1 in range(sqind[0], sqind[0] + SquareSize):
                for ind2 in range(sqind[1], sqind[1] + SquareSize):
                    if(B[ind1,ind2] == val):
                        debug('found {} in a square at pos {} {}'.format(val, j, ind))
                        found = True
                        
            # Value hasn't been found so it can be a possible one.
            if not found:
                debug('{} not found, added to possibles...'.format(val))
                poss = np.insert(poss, poss.size, val)
        debug('Possible values for {} {} : {}'.format(i,j,poss))
        return poss;
    for i in range(BoardLength):
        for j in range(BoardLength):
            if (B[i,j] == 0):
                p_array = getPossibles(i,j)
                for p in p_array:
                    B[i,j] = p
                    if THINKING:
                        debug('*** Trying {} {} with value {}'.format(i,j,p))
                        DisplayBoard(B,OriginalBoard)
                    if Solve(B) == True:
                        return True
                    else:
                        B[i,j] = 0
                return False   
    return True    



# MAIN
# board must be square.
BoardSize = Board.size
BoardLength = np.int(np.sqrt(Board.size))
SquareSize = np.int(np.sqrt(BoardLength))

# Keep original Board for colored display (keeping track of new and fixed values)
OriginalBoard = Board.copy()

DisplayBoard(Board)

from timeit import default_timer as timer
start = timer()

Solve(Board)

end = timer()
print('Total time : {}'.format(end - start))


DisplayBoard(Board, OriginalBoard)


||=====================================||
||   | 5 |   ||   |   |   || 6 | 2 |   || 
||-------------------------------------||
||   |   |   || 2 |   |   || 7 |   |   || 
||-------------------------------------||
|| 1 |   |   || 4 |   |   || 8 | 9 |   || 
||=====================================||
||   |   |   ||   | 2 |   || 9 |   | 6 || 
||-------------------------------------||
||   | 8 |   || 3 | 4 | 6 ||   | 5 |   || 
||-------------------------------------||
|| 6 |   | 1 ||   | 9 |   ||   |   |   || 
||=====================================||
||   | 4 | 6 ||   |   | 3 ||   |   | 9 || 
||-------------------------------------||
||   |   | 7 ||   |   | 2 ||   |   |   || 
||-------------------------------------||
||   | 9 | 8 ||   |   |   ||   | 7 |   || 
||=====================================||
Total time : 2.800711558101284

||=====================================||
|| 7 | 5 | 4 || 1 | 8 | 9 || 6 | 2 | 3 || 
||-------------------------------------||
|| 8 | 6 | 9 || 2 | 3 | 5 || 7 | 